In [ ]:
import torch
import math
import os
import matplotlib.pyplot as plt
from torch import optim
import torch._dynamo
from torchvision import models
from torch.profiler import profile, record_function, ProfilerActivity

pi = math.pi
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [ ]:
def fn(x):
    return torch.sin(x)**2 + torch.cos(x)**2

In [ ]:
torch.manual_seed(0)
x = torch.rand(1000000, requires_grad=True).to(device)

out = fn(x)
torch.linalg.norm(out-1)

In [ ]:
torch.manual_seed(0)
x = torch.rand(1000000, requires_grad=True).to(device)

def inspect_backend(gm, sample_inputs):
    gm.print_readable()
    return gm.forward

torch._dynamo.reset()
compiled_model = torch.compile(fn, backend=inspect_backend)

out = compiled_model(x)

In [ ]:
import torch._dynamo
from torch.fx.passes.graph_drawer import FxGraphDrawer
from torch._functorch.aot_autograd import aot_module_simplified

def inspect_backend(gm, sample_inputs): 
    def fw(gm, sample_inputs):
        gm.print_readable()
        g = FxGraphDrawer(gm, 'fn')
        with open("forward.svg", "wb") as f:
            f.write(g.get_dot_graph().create_svg())
        return gm.forward
    
    def bw(gm, sample_inputs):
        gm.print_readable()
        g = FxGraphDrawer(gm, 'fn')
        with open("backward.svg", "wb") as f:
            f.write(g.get_dot_graph().create_svg())
        return gm.forward

    # Invoke AOTAutograd
    return aot_module_simplified(
        gm,
        sample_inputs,
        fw_compiler=fw,
        bw_compiler=bw
    )

torch._dynamo.reset()
compiled_model = torch.compile(fn, backend=inspect_backend)

out = compiled_model(x).sum().backward()


In [ ]:
compiled_model = torch.compile(fn, backend='inductor',
                              options={'trace.enabled':True,
                                      'trace.graph_diagram':True})

out = compiled_model(x).sum().backward()